In [19]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys

from importlib import reload


sys.path.append("D:/DST/")
import cathay_db as db
import utils as ut
import financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

In [2]:
df_delisted = ut.concatenate_excel_sheets('./data/tej_財報_2000_2023_下市.xlsx')
df_listed_IFRS = ut.concatenate_excel_sheets('./data/tej_IFRS_2005_2023_上市.xlsx')
df_listed_Finance_DB = ut.concatenate_excel_sheets('./data/tej_FinanceDB_2000_2019_上市.xlsx')
df_listed_IFRS_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_cum_2005_2023_上市.xlsx')

df_delisted_lsd = ut.concatenate_excel_sheets('./data/tej_長短期借款_2000_2023_下市.xlsx')
df_delisted_lsd = ut.tej_data_clean_up(df_delisted_lsd)
df_delisted_lsd.drop_duplicates(inplace=True)

df_delisted = ut.tej_data_clean_up(df_delisted)
df_listed_IFRS = ut.tej_data_clean_up(df_listed_IFRS)
df_listed_Finance_DB = ut.tej_data_clean_up(df_listed_Finance_DB)
df_listed_IFRS_cum = ut.tej_data_clean_up(df_listed_IFRS_cum)

df_delisted.drop_duplicates(inplace=True)
df_listed_IFRS.drop_duplicates(inplace=True)
df_listed_Finance_DB.drop_duplicates(inplace=True)
df_listed_IFRS_cum.drop_duplicates(inplace=True)

df_delisted.columns = [col[1:].strip() if col[0] == 'Q' else col.strip() for col in df_delisted.columns]
df_listed_IFRS.columns = [col.strip() for col in df_listed_IFRS.columns]
df_listed_Finance_DB.columns = [col[1:].strip() if col[0] == 'M' else col.strip() for col in df_listed_Finance_DB.columns]

# to feather
df_delisted.reset_index(drop=True).to_feather('./data/tej_財報_2000_2023_下市.feather')
df_listed_IFRS.reset_index(drop=True).to_feather('./data/tej_IFRS_2005_2023_上市.feather')
df_listed_Finance_DB.reset_index(drop=True).to_feather('./data/tej_FinanceDB_2000_2019_上市.feather')
df_listed_IFRS_cum.reset_index(drop=True).to_feather('./data/tej_IFRS_cum_2005_2023_上市.feather')

In [2]:
# from feather
df_delisted = pd.read_feather('./data/tej_財報_2000_2023_下市.feather')
df_listed_IFRS = pd.read_feather('./data/tej_IFRS_2005_2023_上市.feather')
df_listed_Finance_DB = pd.read_feather('./data/tej_FinanceDB_2000_2019_上市.feather')
df_listed_IFRS_cum = pd.read_feather('./data/tej_IFRS_cum_2005_2023_上市.feather')

In [7]:
df_delisted = df_delisted.merge(df_delisted_lsd[['股票代號','年月','長短期借款']], on=['股票代號', '年月'], how='left')
df_listed_Finance_DB['長短期借款'] = np.nan

df_listed_Finance_DB['sequence'] = 1
df_listed_Finance_DB['財報發布日'] = np.nan
df_listed_IFRS_cum['sequence'] = 2
df_delisted['sequence'] = 3
df_delisted['財報發布日'] = np.nan


In [18]:
df_delisted_columns = pd.DataFrame(df_delisted.columns, columns=['delisted'])
df_listed_IFRS_cum_columns = pd.DataFrame(df_listed_IFRS_cum.columns, columns=['IFRS'])
df_listed_Finance_DB_columns = pd.DataFrame(df_listed_Finance_DB.columns, columns=['Finance_DB'])

# merge all df to check the columns
df_all_columns = pd.merge(df_delisted_columns, df_listed_IFRS_cum_columns, how='outer', left_on='delisted', right_on='IFRS')
df_all_columns = pd.merge(df_all_columns, df_listed_Finance_DB_columns, how='outer', left_on='delisted', right_on='Finance_DB')


# drop the columns that only Finance_DB has na
df_all_columns = df_all_columns.dropna(how = 'any')
common_column = df_all_columns['delisted'].tolist()

In [11]:
df_fs_all = pd.DataFrame([])
# concat the all df
df_fs_all = pd.concat([df_fs_all, df_listed_IFRS[common_column]],axis=0)
df_fs_all = pd.concat([df_fs_all, df_listed_Finance_DB[common_column]],axis=0)
df_fs_all = pd.concat([df_fs_all, df_delisted[common_column]],axis=0)

df_seq = df_fs_all.groupby(['股票代號','年月'])['sequence'].min().reset_index().copy()
df_fs_merge = pd.merge(df_seq, df_fs_all, how='left', on=['股票代號','年月','sequence'])

df_fs_merge.sort_values(['股票代號','年月'], inplace=True)
# filter out the data that '股票代號' length is not 4
df_fs_merge = df_fs_merge[df_fs_merge['股票代號'].str.len() == 4]
df_fs_merge.reset_index(drop=True, inplace=True)
df_fs_merge.to_feather('./data/df_tej_fs_merge.feather')

In [86]:
df_fs_merge = pd.read_feather('./data/df_tej_fs_merge.feather')

In [84]:
df_slice_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_slice_cum_2000_2023_All.xlsx')
del df_slice_cum['來自營運之現金流量']

In [85]:
del df_slice_cum['繼續營業單位損益']

In [87]:
df_slice_cum = pd.merge(df_slice_cum, df_fs_merge[['公司','年月','來自營運之現金流量']], how='left', on=['公司','年月'])

In [39]:
df_slice_cum = ut.tej_data_clean_up(df_slice_cum)

In [94]:
df_slice_ni_cum = ut.concatenate_excel_sheets('./data/tej_IFRS_ni_slice_cum_2000_2023_All.xlsx')
df_slice_ni_cum = ut.tej_data_clean_up(df_slice_ni_cum)
df_slice_ni_cum.sort_values(['公司','年月'], inplace=True)
df_slice_ni_cum['last_date'] = df_slice_ni_cum.groupby('股票名稱')['年月'].shift(1)
# replace the last_date to np.nan if the last_date is not the same year with 年月
df_slice_ni_cum.loc[df_slice_ni_cum['last_date'].astype('str').str[:4] != df_slice_ni_cum['年月'].astype('str').str[:4], 'last_date'] = np.nan
df_slice_ni_cum = df_slice_ni_cum.merge(df_slice_ni_cum[['公司','年月','歸屬母公司淨利（損）']], how='left', left_on=['公司','last_date'], right_on=['公司','年月'], suffixes=('', '_y'))
df_slice_ni_cum['ni'] = df_slice_ni_cum['歸屬母公司淨利（損）']  - df_slice_ni_cum['歸屬母公司淨利（損）_y']
df_slice_ni_cum['ni'] = np.where(df_slice_ni_cum['ni'].isna(), df_slice_ni_cum['歸屬母公司淨利（損）'], df_slice_ni_cum['ni'])

In [95]:
df_slice_cum = pd.merge(df_slice_cum, df_slice_ni_cum[['公司','年月','ni']], how='left', on=['公司','年月'])

In [97]:
df_slice_cum.sort_values(['公司','年月'], inplace=True)

In [99]:
df_slice_cum.reset_index(drop=True, inplace=True)

In [101]:
df_slice_cum.to_feather('./data/df_tej_fs_slice_cum.feather')

In [102]:
ut.CDF(df_slice_ni_cum).find_ticker('2330')

,股票名稱,股票代號,公司,年月,歸屬母公司淨利（損）,last_date,年月_y,歸屬母公司淨利（損）_y,ni
23283,台積電,2330,2330 台積電,1999-12-28,23526994.0000,NaT,NaT,NaN,23526994.0000
23284,台積電,2330,2330 台積電,2000-12-30,65106194.0000,NaT,NaT,NaN,65106194.0000
23285,台積電,2330,2330 台積電,2001-12-31,14483174.0000,NaT,NaT,NaN,14483174.0000
23286,台積電,2330,2330 台積電,2002-12-31,21610291.0000,NaT,NaT,NaN,21610291.0000
23287,台積電,2330,2330 台積電,2003-12-31,47258700.0000,NaT,NaT,NaN,47258700.0000
23288,台積電,2330,2330 台積電,2004-12-31,92316115.0000,NaT,NaT,NaN,92316115.0000
23289,台積電,2330,2330 台積電,2005-06-30,35187334.0000,NaT,NaT,NaN,35187334.0000
23290,台積電,2330,2330 台積電,2005-12-30,93575035.0000,2005-06-30,2005-06-30,35187334.0000,58387701.0000
23291,台積電,2330,2330 台積電,2006-06-30,66608628.0000,NaT,NaT,NaN,66608628.0000
23292,台積電,2330,2330 台積電,2006-12-29,127009731.0000,2006-06-30,2006-06-30,66608628.0000,60401103.0000
